# Activity 09
## Tyler Barna

### Contents
1. [Import Modules and Data](#import)
2. [Visualizing Data](#data)  
3. [Analysis](#analysis)    
    3.1 [Analysis 1](#analysis1)

### 1. Import Modules and Data
<a id='import'></a>

In [10]:
import csv
import dis
import inspect
import os
import sys
import time

import arviz as az
import astropy
import astroquery
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import pymc as pm
import random
import scipy as sp
import scipy.stats as stats
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

from astropy import units as u
from astropy import constants as const
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.io import ascii
from astropy.time import Time
from astropy.time import TimeDelta
from astropy.timeseries import TimeSeries
from astropy.visualization import time_support
time_support()
from astropy.visualization import quantity_support
quantity_support()

from IPython.display import display_html
from IPython.display import Image

from numpy import interp

from pymc import Model, Normal, Gamma, find_MAP

from scipy import integrate
from scipy import linalg as la
from scipy import optimize

from scipy.stats import beta
from scipy.stats import betabinom
from scipy.stats import binom
from scipy.stats import gamma
from scipy.stats import invgamma
from scipy.stats import multivariate_normal as mvn
from scipy.stats import nbinom
from scipy.stats import norm
from scipy.stats import poisson
from scipy.stats import t
from scipy.stats import uniform

from sklearn.linear_model import LinearRegression as linreg
from sklearn import preprocessing as preproc

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

pp = pprint.PrettyPrinter(indent=4)

## set seed for reproducibility
random.seed(5731)

## import data

df = pd.read_csv('Galaxy_Sample_Logistic.csv')
display(df.describe())

,Morph,Density,Absolute
count,4355.000000,4355.000000,4355.000000
mean,0.251435,0.134801,-20.305943
std,0.433888,0.715290,0.786539
min,0.000000,-1.495605,-22.819994
25%,0.000000,-0.383498,-20.864103
50%,0.000000,0.046150,-20.261288
75%,1.000000,0.600258,-19.668709
max,1.000000,2.740877,-19.000109


### 2. Visualizing Data
<a id='data'></a>

### 3. Analysis
<a id='analysis'></a>

#### 3.1 Analysis 1
<a id='analysis1'></a>